In [ ]:
import pandas as pd
import folium
from collections import Counter
from pathlib import Path

In [7]:
gtfs_path = Path("..") / "data" / "raw" / "budapest_gtfs"

In [8]:
# Load GTFS core files
stops = pd.read_csv(gtfs_path / "stops.txt")
routes = pd.read_csv(gtfs_path / "routes.txt")
trips = pd.read_csv(gtfs_path / "trips.txt")
stop_times = pd.read_csv(gtfs_path / "stop_times.txt")

In [9]:
# Clean data
stops = stops.dropna(subset=["stop_lat", "stop_lon"])
stops["stop_name"] = stops["stop_name"].str.strip()

In [10]:
# Map all tram/bus stops in Budapest
budapest_map = folium.Map(location=[47.4979, 19.0402], zoom_start=12)
for _, row in stops.iterrows():
    folium.CircleMarker(
        location=[row["stop_lat"], row["stop_lon"]],
        radius=2,
        popup=row["stop_name"],
        color="blue",
        fill=True,
        fill_opacity=0.5
    ).add_to(budapest_map)

In [ ]:
# Find a specific route, e.g., line 4-6 tram
route_ids = routes[routes["route_short_name"].isin(["4", "6"])]["route_id"].tolist()
trips_4_6 = trips[trips["route_id"].isin(route_ids)]
stop_ids_4_6 = stop_times[stop_times["trip_id"].isin(trips_4_6["trip_id"])]["stop_id"].unique()
stops_4_6 = stops[stops["stop_id"].isin(stop_ids_4_6)]

In [12]:
# Add 4-6 tram stops to map in red
for _, row in stops_4_6.iterrows():
    folium.CircleMarker(
        location=[row["stop_lat"], row["stop_lon"]],
        radius=3,
        popup=row["stop_name"],
        color="red",
        fill=True,
        fill_opacity=0.8
    ).add_to(budapest_map)

In [13]:
# Identify most connected stops (hubs) by counting stop_id occurrences in stop_times
stop_counts = Counter(stop_times["stop_id"])
hub_data = pd.DataFrame(stop_counts.items(), columns=["stop_id", "count"]).sort_values(by="count", ascending=False)
hubs = stops[stops["stop_id"].isin(hub_data.head(10)["stop_id"])]

In [14]:
# Add hubs to map in green
for _, row in hubs.iterrows():
    folium.CircleMarker(
        location=[row["stop_lat"], row["stop_lon"]],
        radius=5,
        popup=f"{row['stop_name']} (hub)",
        color="green",
        fill=True,
        fill_opacity=1.0
    ).add_to(budapest_map)

In [23]:
budapest_map.save("../data/budapest_transport_map.html")